# Summary

`# TODO: summary here`

In [1]:
%load_ext autoreload
%autoreload 3

In [4]:
import gc
import inspect
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/incendio


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
print(
"""
# OPTION 1
@auto_cli()
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
    
    
# OPTION 2
@auto_cli(f1, f2)
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
"""
)


# OPTION 1
@auto_cli()
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
    
    
# OPTION 2
@auto_cli(f1, f2)
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)



## isstatic

In [80]:
from inspect import ismethod, signature
from types import FunctionType, MethodType

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
def bar():
    return 'in bar'

In [114]:
def baz(cls):
    return 'in baz'

In [135]:
def bak(unk):
    return 'in bak'

In [137]:
class Foo:
    
    class_var = 'my_class_var'
    
    def __init__(self, inst_var='my_inst_var'):
        self.inst_var = inst_var
        self.inst_var_func = bar
        self.inst_var_func_baz = baz
        self.inst_var_func_bak = bak
        
    def my_method(self, x):
        return 'in regular method'
    
    @staticmethod
    def my_static(y):
        return 'in staticmethod', y
    
    @classmethod
    def my_classmethod(cls, *args):
        return cls('-'.join(args))
    
    @abstractmethod
    def my_abstract(self):
        raise NotImplementedError()
        
    @property
    def my_prop(self):
        return 'my_prop_val'

In [138]:
f = Foo()

In [139]:
f.inst_var_func

<function __main__.bar()>

In [140]:
f.my_static

<function __main__.Foo.my_static(y)>

In [141]:
hdir(Foo)

{'class_var': 'attribute',
 'my_abstract': 'method',
 'my_classmethod': 'method',
 'my_method': 'method',
 'my_prop': 'attribute',
 'my_static': 'method'}

In [142]:
hdir(f)

{'class_var': 'attribute',
 'inst_var': 'attribute',
 'inst_var_func': 'method',
 'inst_var_func_bak': 'method',
 'inst_var_func_baz': 'method',
 'my_abstract': 'method',
 'my_classmethod': 'method',
 'my_method': 'method',
 'my_prop': 'attribute',
 'my_static': 'method'}

In [143]:
type(f.my_static)

function

In [146]:
def _isstatic(meth):
    # subcase: assume we've already checked that meth is callable. Also assume
    # this is coming from an instance, not a class. Logic is not airtight yet:
    # better not to rely on param names.
    if ismethod(meth): 
        return False
    params_ = list(params(meth))
    if params_ and params_[0] not in {'cls', 'self'}:
        return True
    return False

In [145]:
for k, v in hdir(f).items():
    if v == 'attribute': continue
    print(k, v)
    obj = getattr(f, k)
    print(obj)
    print(type(obj))
    print('qualname dots', getattr(obj, '__qualname__', '').count('.'))
    print('is method:', ismethod(obj))
    print('is FunctionType:', isinstance(obj, FunctionType))
    print('is MethodType:', isinstance(obj, MethodType))
    try:
        print(list(params(obj)))
    except Exception as e:
        print(e)    
    print('is static', _isstatic(obj))
    print()

inst_var_func method
<function bar at 0x1a237e09d8>
<class 'function'>
qualname dots 0
is method: False
is FunctionType: True
is MethodType: False
[]
[]
is static False

inst_var_func_bak method
<function bak at 0x1a22bc39d8>
<class 'function'>
qualname dots 0
is method: False
is FunctionType: True
is MethodType: False
['unk']
cls or self para name
is static True

inst_var_func_baz method
<function baz at 0x1a23104ea0>
<class 'function'>
qualname dots 0
is method: False
is FunctionType: True
is MethodType: False
['cls']
['cls']
is static False

my_abstract method
<bound method Foo.my_abstract of <__main__.Foo object at 0x1a23630898>>
<class 'method'>
qualname dots 1
is method: True
is FunctionType: False
is MethodType: True
[]
_isstatic condition ismethod
is static False

my_classmethod method
<bound method Foo.my_classmethod of <class '__main__.Foo'>>
<class 'method'>
qualname dots 1
is method: True
is FunctionType: False
is MethodType: True
['args']
_isstatic condition ismethod
is st

In [128]:
'bound' in str(f.my_classmethod)

True

In [108]:
for k, v in hdir(Foo).items():
    print(k, v)
    obj = getattr(Foo, k)
    print(type(obj))
    print('is method:', ismethod(obj))
    print('is FunctionType:', isinstance(obj, FunctionType))
    print('is MethodType:', isinstance(obj, MethodType))
    try:
        print(list(params(obj)))
    except Exception as e:
        print(e)
    print()

class_var attribute
<class 'str'>
is method: False
is FunctionType: False
is MethodType: False
'my_class_var' is not a callable object

my_abstract method
<class 'function'>
is method: False
is FunctionType: True
is MethodType: False
['self']

my_classmethod method
<class 'method'>
is method: True
is FunctionType: False
is MethodType: True
['args']

my_method method
<class 'function'>
is method: False
is FunctionType: True
is MethodType: False
['self', 'x']

my_prop attribute
<class 'property'>
is method: False
is FunctionType: False
is MethodType: False
<property object at 0x1a2351b278> is not a callable object

my_static method
<class 'function'>
is method: False
is FunctionType: True
is MethodType: False
['y']



In [65]:
list(params(Foo.my_static))

['y']

In [56]:
staticmethod??

## Override_self

Thinking out loud: we can't use add_kwargs here to update the method's signature because we don't know what attributes self has until the method is called (we get them by examining args[0] in wrapper). Any way around this would probably be extremely hacky. I suppose if we mandated `keep` we could add those ahead of time but we wouldn't be able to show defaults.

In [241]:
from contextlib import contextmanager

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [153]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    def call(self, **kwargs):
        a = kwargs.pop('a', self.a)
        b = kwargs.pop('b', self.b)
        c = kwargs.pop('c', self.c)
        return a, b, c

In [154]:
t = Tree(1, 2, 3)
t

Tree(a=1, b=2, c=3)

In [155]:
t.call()

(1, 2, 3)

In [156]:
t.call(c=44)

(1, 2, 44)

@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @kwargs_fallback
    def call(self, **kwargs):
        # Signature shows a, b, and c with class default values.
        return a, b, c

In [242]:
@contextmanager
def temporary_global_scope(kwargs):
    # Make kwargs temporarily available as global vars.
    old_globals = globals().copy()
    globals().update(kwargs)
    try:
        yield
    finally:
        for k in kwargs:
            if k in old_globals:
                globals()[k] = old_globals[k]
            else:
                del globals()[k]

In [435]:
# diff name for now since htools has other kwargs_fallback func.
def fallback(meth=None, *, keep=(), drop=(), save=False):
    """Make instance/class attributes available as default arguments for a 
    method. Kwargs can be passed in to override one or more of them. You can
    also choose to use kwargs to update the instance attributes if desired.
    
    Using the decorator without parentheses will use the default values for
    keep/drop/save. If you want to specify arguments, they must be passed in
    as keyword args (meth is never explicitly passed in, of course).
    
    Parameters
    ----------
    meth: method
        The method to decorate. Unlike the other arguments, this is passed in
        implicitly.
    keep: Iterable[str] or str
        Name(s) of instance attributes to include. If you specify a value 
        here, ONLY these instance attributes will be made available as 
        fallbacks. If you don't pass in any value, the default is for all
        instance attributes to be made available. You can specify `keep`, 
        `drop`, or neither, but not both. This covers all possible options:
        keep only a few, keep all BUT a few, or keep all (drop all is the
        default case and doesn't require a decorator).
    drop: Iterable[str] or str
        Name(s) of instance attributes to ignore. I.e. if you want to make
        all instance attributes available as fallbacks except for self.df,
        you could specify drop=('df').
    save: bool
        If True, kwargs that share names with instance attributes will be
        overwritten with their new values. E.g. if we previously had
        self.lr = 3e-3 and you call your decorated method with 
        obj.mymethod(lr=1), self.lr will be set to 1.
    """
    if meth is None: 
        # Want to avoid errors if user passes in string or leaves comma out of
        # tuple when specifying keep/drop.
        return partial(fallback, keep=tolist(keep), drop=tolist(drop), 
                       save=save)
    
    @wraps(meth)
    def wrapper(*args, **kwargs):
        self = args[0]
        self_kwargs = vars(self)
        if keep or drop: self_kwargs = select(self_kwargs, keep, drop)

        # Update kwargs with instance attribute defaults. Also update self if
        # user asked to save kwargs.
        for k, v in self_kwargs.items():
            if k not in kwargs:
                kwargs[k] = v
            elif save:
                setattr(self, k, kwargs[k])
                
        # Execute and return.
        with temporary_global_scope(kwargs):
            return meth(*args, **kwargs)
    return wrapper

In [436]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback(save=True)
    def call(self, **kwargs):
        return a, b, c

In [437]:
t = Tree(1, 2, 3)
t.call(b='bbb')

(1, 'bbb', 3)

In [438]:
assert t.b == 'bbb'

In [439]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback(save=False)
    def call(self, **kwargs):
        return a, b, c

In [440]:
t = Tree(1, 2, 3)
t.call(b='bbb')

(1, 'bbb', 3)

In [441]:
assert t.b == 2

In [442]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback
    def call(self, **kwargs):
        return a, b, c

In [443]:
t = Tree(1, 2, 3)
t.call(b='bbb')

(1, 'bbb', 3)

In [444]:
assert t.b == 2

In [445]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback(keep=('a'))
    def call(self, **kwargs):
        return a, b, c

In [446]:
t = Tree(1, 2, 3)
with assert_raises(NameError):
    t.call(a='aaa', b='bbb')

As expected, got NameError(name 'c' is not defined).


In [447]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback(drop=('c'))
    def call(self, **kwargs):
        return a, b

In [448]:
t = Tree(1, 2, 3)
t.call(b='bbb')

(1, 'bbb')

In [450]:
assert t.b == 2